In [107]:
#dissolve by district and province - done
#cut by new district and province
#calculate area
#zonal stats on pop, build
#reproject back to 4326 and save in db

In [124]:
import os
import geopandas as gpd
import pandas as pd
import pyproj
from sqlalchemy import create_engine
import psycopg2 # required for exporting to postgis
import folium
from folium.plugins import MousePosition
import rioxarray as rxr

import matplotlib.pyplot as plt
#from sqlalchemy import create_engine
#import pyproj
#import pandas as pd
#import numpy as np

In [125]:
password = 'pewp7re'
db_connection_url = "postgresql://postgres:{}@localhost:5432/HSDC".format(password)
con = create_engine(db_connection_url)

In [126]:
pd.set_option('display.max_columns', None)

In [127]:
os.chdir(r'D:\iMMAP\proj\ASDC\data\flood_layer')

In [128]:
admin_level_column = 'adm2_pcode'

## Preping vector data

In [129]:
# Define projection
repro_crs = '+proj=cea'

# Load admin
admin = gpd.GeoDataFrame.from_postgis('SELECT * from afg_admbnda_adm2', con).to_crs(repro_crs)

In [130]:
# Load dissolved flood layers
#flood_l1 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_029cm_dissolve_all_v2', con, 'wkb_geometry') #.to_crs(repro_crs)
#flood_l2 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_121cm_dissolve_all_v2', con, 'wkb_geometry') #.to_crs(repro_crs)
flood_l3 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_271cm_dissolve_all_v2', con, 'wkb_geometry').to_crs(repro_crs)


In [132]:
# Clip flood areas from admin polygons
flood = admin.clip(flood_l3).to_crs('epsg:4326')
flood.to_postgis("afg_fldzonea_100k_ncia_v2_271_adm2", con)#

In [116]:
# Clip flood areas from admin polygons
#flood = admin.clip(flood_l2)
#flood.to_postgis("afg_fldzonea_100k_ncia_v2_121_adm2", con)#

In [117]:
# Clip flood areas from admin polygons
#flood = admin.clip(flood_l1)
#flood.to_postgis("afg_fldzonea_100k_ncia_v2_029_adm2", con)#

In [135]:
# Load flood zones clipped with admin
#flood_l1 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_029cm_adm2', con, 'wkb_geometry').to_crs(repro_crs)
#flood_l2 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_121cm_adm2', con, 'wkb_geometry').to_crs(repro_crs)
flood_l3 = gpd.GeoDataFrame.from_postgis('SELECT * from afg_fldzonea_100k_ncia_v2_271_adm2', con, 'geom').to_crs(repro_crs)

In [136]:
flood = flood_l3

In [ ]:

#water_shed = gpd.GeoDataFrame.from_postgis('SELECT * from afg_sheda_lvl3', con, 'wkb_geometry').to_crs(repro_crs)
#pop = rio.open(r'D:\iMMAP\data\Afghanistan\afg_worldpop_2020_UNadj_unconstrained.tif')

### Extracting attributes

In [138]:
# Calculating area and converting from square m to square km
flood['area_km2'] = flood['geom'].area.div(1000000)   ## OBS: CHANGE geometry to geom for postgres!

In [139]:
# Load building centroids
build_centroids = gpd.GeoDataFrame.from_postgis('SELECT * from afg_buildings_microsoft_centroids', con).to_crs(repro_crs)

In [140]:
# Join building centroids to flood polygons
joined_df = gpd.sjoin(
    build_centroids,
    flood,
    how='inner',
    op='intersects'
)

# Count number of buildings within admin polygons
build_count = joined_df.groupby(
    [admin_level_column],
    as_index=False,
)['geom'].count() # vuid is arbitrary, we just count the rows

# Change column name to build_count
build_count.rename(columns = {'geom': 'build_count'}, inplace = True)

C:\Users\VMO\anaconda3\envs\earth2\lib\site-packages\IPython\core\interactiveshell.py:3301: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [141]:
# Merge build count back on to admin dataset
flood = flood.merge(
    build_count, 
    on=admin_level_column, 
    how='left')

In [142]:
# Check total number of buildings within flood zones
flood['build_count'].sum()

258382.0

In [ ]:
flood = flood.to_crs('epsg:4326')

In [143]:
flood.to_postgis("afg_fldzonea_100k_ncia_v2_271_adm2-area-build", con)#

In [144]:
flood_area_build = gpd.GeoDataFrame.from_postgis("afg_fldzonea_100k_ncia_v2_271_adm2-area-build", con)


CRSError: Invalid projection: epsg:54034: (Internal Proj Error: proj_create: crs not found)

### Extracting pop

In [ ]:
pop_path = r'D:\iMMAP\data\Afghanistan\afg_worldpop_2020_UNadj_unconstrained.tif'
pop_rxr = rxr.open_rasterio(pop_path, masked=True).squeeze() # Open with rxr, much easier to reproject